In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host ='localhost',
    database= 'retail_analysis',
    user = 'postgres',
    password ='1234#',
    port ='5432'
)

df = pd.read_sql('SELECT * FROM products;', conn)
print(df.head())